In [4]:
# Standard library
import os
import datetime

# Third-party libraries
import ee
import geemap
import requests
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.signal import savgol_filter
from osgeo import gdal

# Jupyter / interactive widgets
import ipywidgets as widgets
from ipywidgets import HTML
from ipyleaflet import WidgetControl


In [5]:
# Start session in GEE 
ee.Authenticate()

print("Insert your name project from GEE:")
project_name = input().strip()

project = project_name
ee.Initialize(project=project)

Insert your name project from GEE:


In [6]:
# Before fire date range
pre_start_date = ee.Date('2025-01-01')
pre_end_date = pre_start_date.advance(4, 'day')  # 2025-01-05

# After fire date range
pos_start_date = pre_end_date.advance(1, 'day')  # 2025-01-06
pos_end_date = pos_start_date.advance(10, 'day')  # 2025-01-16

# Coordinates of interest
latitude = 34.092615
longitude = -118.532875

# Region of interest (ROI) with a buffer of 15km
roi = ee.Geometry.Point(longitude, latitude).buffer(15000)


In [7]:
# Define palette color for visualization (RGB, NDVI and NBR) and color bar settings

rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  
    'gamma': 1.2
}


ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
    
}

diff_nbr_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']
}

#Initialize map with center and zoom
Map = geemap.Map(center=[latitude, longitude], zoom=13)


In [8]:
# Define functions

def get_all_collections(start_date: str, end_date: str) -> dict:

    """
    Generate median Sentinel-2 composites (reflectance, NDVI, and NBR)
    for a specified date range.

    Parameters
    ----------
    start_date : str
        Start date in 'YYYY-MM-DD' format.
    end_date : str
        End date in 'YYYY-MM-DD' format.

    Returns
    -------
    dict
        Dictionary containing:
        - 'image' : ee.Image
            Median reflectance composite.
        - 'ndvi' : ee.Image
            Median NDVI image.
        - 'nbr' : ee.Image
            Median NBR image.
    """

    try:
        # Selecting Sentinel-2 collection with defined date range, ROI and cloud cover filter
        collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

        # Generating median reflectance composite
        image = collection.select('B8', 'B12', 'B4', 'B3', 'B2').median().clip(roi)

        # Generating median NDVI composite
        ndvi = collection.select(['B8', 'B4']) \
            .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
            .median().clip(roi)
        
        # Generating median NBR composite
        nbr = collection.select(['B8', 'B12']) \
            .map(lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR')) \
            .median().clip(roi)

        return {'image': image, 'ndvi': ndvi, 'nbr': nbr}

    except Exception as e:
        print(f"Error: {e}")
        

# Export function to Google Drive
def export_to_drive(image: ee.Image, image_title: str, folder_name='GEE_exports'):

    """
    Export an Earth Engine image to Google Drive.

    Parameters
    ----------
    image : ee.Image
        Image to export.
    image_title : str
        Title for the exported image.
    folder_name : str, optional
        Name of the Google Drive folder where the image will be saved.
    """

    try:
        # Initializing export task to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=image_title,
            folder=folder_name,
            fileNamePrefix=image_title.lower().replace(' ', '_'),
            scale=10,
            region=roi,
            fileFormat='GeoTIFF',
            maxPixels=1e10
        )

        task.start()
        
        print(
            f'Started export task for {image_title}. '
            f'Check your Google Drive folder "{folder_name}" when complete.'
        )

    except Exception as e:
        print(f"Error: {e}")


# Create HTML labels for map output
def create_labels_html(legend_before: str, legend_after: str) -> tuple:

    """
    Create HTML label elements for map display.

    Parameters
    ----------
    legend_before : str
        Text for the label positioned in the top-left corner.
    legend_after : str
        Text for the label positioned in the top-right corner.

    Returns
    -------
    tuple
        Tuple of IPython.display.HTML objects:
        (before_label, after_label), styled for use in folium/geemap maps.
    """

    before_html = HTML(
        f"""<div style="position: absolute; top: 10px; left: 10px; z-index: 1000;
        background-color: white; padding: 5px; border-radius: 5px;
        font-weight: bold; color: black;">{legend_before}</div>"""
    )

    after_html = HTML(
        f"""<div style="position: absolute; top: 70px; right: 10px; z-index: 1000;
        background-color: white; padding: 5px; border-radius: 5px;
        font-weight: bold; color: black;">{legend_after}</div>"""
    )

    return before_html, after_html


# Get the most recent date of available images
def get_most_recent_date(start_date: str) -> str:

    """
    Retrieve the most recent available Sentinel-2 acquisition date
    after a given start date.

    Parameters
    ----------
    start_date : str
        Start date in 'YYYY-MM-DD' format.

    Returns
    -------
    str
        Most recent date in 'YYYY-MM-DD' format.
    """
    
    try:
        # Gets today's date
        today = datetime.date.today().isoformat()


        # Selecting Sentinel-2 collection with defined date range, ROI and cloud cover filter
        collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, today) \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
            .sort('system:time_start', False)

        # Getting the most recent image from the collection
        most_recent_image = collection.first()

        # Getting the date of the most recent image
        most_recent_date = ee.Date(
            most_recent_image.get('system:time_start')
        ).format('YYYY-MM-dd')

        return most_recent_date.getInfo()

    except Exception as e:
        print(f"Error in get_most_recent_date: {e}")


In [9]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN RGB

map_rgb = Map

# Pre and post fire images in RGB
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['image']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']


# Add split map to compare before and after fire images in RGB
left_layer = geemap.ee_tile_layer(pre_image, rgb_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')

map_rgb.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_rgb.addLayer(pre_image, rgb_vis, 'Before Fire', False)
map_rgb.addLayer(pos_image, rgb_vis, 'After Fire', False)


# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_rgb.add_control(WidgetControl(widget=before_html, position='topleft'))
map_rgb.add_control(WidgetControl(widget=after_html, position='topright'))

map_rgb



#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================

# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")




Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [10]:
def download_gee_tiff(image, filename="output.tif", scale=30):

    """
    Download a GeoTIFF image from Google Earth Engine.

    Parameters
    ----------
    image : ee.Image
        Earth Engine image to download.
    filename : str, optional
        Name of the output TIFF file.
    scale : int, optional
        Spatial resolution in meters.

    Returns
    -------
    str
        Path to the downloaded TIFF file.
    """

    # Creates directory if it doesn't exist
    os.makedirs("results", exist_ok=True)
    filename = os.path.join("results", filename)

    # Generates download URL for the image
    url = image.getDownloadURL({
        'scale': scale,
        'format': 'GEO_TIFF'
    })

    print(f"Downloading {filename} ...")
    r = requests.get(url, stream=True)

    # Saves the image to a file with limited memory usage
    with open(filename, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

    print(f"Saved TIFF: {filename}")
    return filename


def tif_to_pdf(tif_path, pdf_path, gamma=1.3, do_percentile_stretch=True):

    """
    Convert a TIFF image to a PDF with optional percentile stretching
    and gamma correction.

    Parameters
    ----------
    tif_path : str
        Path to the input TIFF file.
    pdf_path : str
        Path to the output PDF file.
    gamma : float, optional
        Gamma correction factor.
    do_percentile_stretch : bool, optional
        If True, apply a 2–98 percentile stretch before normalization.

    Returns
    -------
    None
    """
    
    # Creates directory if it doesn't exist
    os.makedirs("results", exist_ok=True)

    tif_path = os.path.join("results", os.path.basename(tif_path))
    pdf_path = os.path.join("results", os.path.basename(pdf_path))


    # Reads TIFF file using GDAL
    ds = gdal.Open(tif_path)
    array = ds.ReadAsArray()

    if array.ndim == 3:
        array = array.transpose((1, 2, 0))
    
    array = array.astype("float32")

    if do_percentile_stretch:
        p2 = np.percentile(array, 2)
        p98 = np.percentile(array, 98)
        array = np.clip((array - p2) / (p98 - p2), 0, 1)
    else:
        arr_min = array.min()
        arr_max = array.max()
        array = (array - arr_min) / (arr_max - arr_min)
        array = np.clip(array, 0, 1)

    array = np.power(array, 1.4 / gamma)

    plt.figure(figsize=(10, 10))
    plt.imshow(array)
    plt.axis("off")
    plt.savefig(pdf_path, bbox_inches='tight', pad_inches=0, dpi=300)
    plt.close()

    print(f"Saved PDF (gamma={gamma}): {pdf_path}")


In [11]:
# Select only RGB bands from Sentinel-2 images
pre_rgb = pre_image.select(['B4', 'B3', 'B2'])
pos_rgb = pos_image.select(['B4', 'B3', 'B2'])


# Download TIFF images 
pre_tif = download_gee_tiff(pre_rgb, "before_fire.tif", scale=30)
pos_tif = download_gee_tiff(pos_rgb, "after_fire.tif", scale=30)


# Converts TIFF to PDF
tif_to_pdf("before_fire.tif", "before_fire.pdf")
tif_to_pdf("after_fire.tif", "after_fire.pdf")


Saved TIFF: results/before_fire.tif
Saved TIFF: results/after_fire.tif
Saved PDF (gamma=1.3): results/before_fire.pdf
Saved PDF (gamma=1.3): results/after_fire.pdf


In [17]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NDVI

map_ndvi = Map


# Pre and post fire images in NDVI
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['ndvi']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['ndvi']


# Add split map to compare before and after fire images in NDVI
left_layer = geemap.ee_tile_layer(pre_image, ndvi_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, ndvi_vis, 'After Fire')

map_ndvi.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_ndvi.addLayer(pre_image, ndvi_vis, 'Before Fire', False)
map_ndvi.addLayer(pos_image, ndvi_vis, 'After Fire', False)


# NDVI color bar
map_ndvi.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    layer_name='NDVI',
    orientation='vertical',  
    position='bottomright'
    )

# Add HTML labels to the map
before_html, adter_html = create_labels_html('Before', 'After')
map_ndvi.add_control(WidgetControl(widget=before_html, position='topleft'))
map_ndvi.add_control(WidgetControl(widget=adter_html, position='topright'))

map_ndvi


#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================


# export_to_drive(pre_image, "before_fire_NDVI_image")
# export_to_drive(pos_image, "after_fire_NDVI_image")


Map(bottom=837395.0, center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [18]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NBR

map_nbr = Map

# Pre and post fire images in NBR
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['nbr']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['nbr']


# Calculate difference between pre and post fire images in NBR
dnbr = pre_image.subtract(pos_image)

# Define a threshold to identify burned areas
burned_mask = dnbr.gt(0.3)

# Calculate total burned area (in hec)
pixel_area = burned_mask.multiply(ee.Image.pixelArea()).divide(10000)  # m^2 to hec
area_burned = pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)


# Calculate area burned in hectares
area_burned_dict = area_burned.getInfo()


# Calculate total area for context
total_area = ee.Image.pixelArea().divide(10000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)

total_area_dict = total_area.getInfo()

# Extract the first (and usually only) value from each dictionary
burned_value = list(area_burned_dict.values())[0]
total_value = list(total_area_dict.values())[0]

print(f"Burned area (hectares): {round(burned_value, 2)}")
print(f"Total area (hectares): {round(total_value, 2)}")
print(f"Burned percentage: {round((burned_value / total_value) * 100, 2)}% of the ROI")

# Add split map to compare before and after fire images in NBR
left_layer = geemap.ee_tile_layer(pre_image, nbr_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, nbr_vis, 'After Fire')


map_nbr.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_nbr.addLayer(pre_image, nbr_vis, 'Before Fire', False)
map_nbr.addLayer(pos_image, nbr_vis, 'After Fire', False)


# NBR color bar
map_nbr.add_colorbar(
    vis_params=nbr_vis,
    label='NBR',
    layer_name='NBR',
    orientation='vertical',
    position='bottomleft'
)

# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_nbr.add_control(WidgetControl(widget=before_html, position='topleft'))
map_nbr.add_control(WidgetControl(widget=after_html, position='topright'))



# Map for difference in NBR
Map_diff = geemap.Map(center=[latitude, longitude], zoom=13)
Map_diff.addLayer(dnbr, diff_nbr_vis, 'ΔNBR')

display(widgets.VBox([Map, Map_diff]))



#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================


# export_to_drive(pre_image, "before_fire_NBR_image")
# export_to_drive(pos_image, "after_fire_NBR_image")
# export_to_drive(dnbr, "Difference NBR")




Burned area (hectares): 9357.08
Total area (hectares): 69822.28
Burned percentage: 13.4% of the ROI


In [14]:
def add_trace(fig, x: list, y: list, name: str, mode='lines+markers',
               line_color='blue', line_width = 2,
                 marker_color='red', marker_size = 8) -> None:
    
    """
    Add a trace to a Plotly figure.

    Parameters
    ----------
    fig : plotly.graph_objs._figure.Figure
        Figure to which the trace will be added.
    x : list
        X-values of the trace.
    y : list
        Y-values of the trace.
    name : str
        Name of the trace.
    mode : str, optional
        Plotly trace mode. Default is 'lines+markers'.
    line_color : str, optional
        Line color. Default is 'blue'.
    line_width : int, optional
        Line width. Default is 2.
    marker_color : str, optional
        Marker color. Default is 'red'.
    marker_size : int, optional
        Marker size. Default is 8.

    Returns
    -------
    None
    """
    
    try:
        # Creates a trace
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode=mode,
            name=name,
            line=dict(color=line_color, width=line_width),
            marker=dict(size=marker_size, color=marker_color),
        ))
    except Exception as e:
        print(f"Error adding trace for {name}: {e}")

def get_ndvi(image):

    """
    Add an NDVI band to an Earth Engine image.

    NDVI is computed as (B8 - B4) / (B8 + B4).

    Parameters
    ----------
    image : ee.Image
        Input Earth Engine image.

    Returns
    -------
    ee.Image
        Image with an added NDVI band.
    """

    # Computes NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def get_nbr(image):

    """
    Add an NBR band to an Earth Engine image.

    NBR is computed as (B12 - B8) / (B12 + B8).

    Parameters
    ----------
    image : ee.Image
        Input Earth Engine image.

    Returns
    -------
    ee.Image
        Image with an added NBR band.
    """
    # Computes NBR
    nbr = image.normalizedDifference(['B12', 'B8']).rename('NBR')
    return image.addBands(nbr)

def ndvi_nbr_plot(start_date:str, end_date:str) -> None:
    
    """
    Plot NDVI and NBR time series for the ROI over a given date range.

    The function computes mean NDVI and NBR values from Sentinel-2 imagery,
    applies Savitzky–Golay smoothing, and saves the result as an interactive
    HTML file.

    Parameters
    ----------
    start_date : str
        Start date in 'YYYY-MM-DD' format.
    end_date : str
        End date in 'YYYY-MM-DD' format.

    Returns
    -------
    None
    """

    try:
        # Load Sentinel-2 SR collection
        collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                        .filterDate(start_date, end_date)
                        .filterBounds(roi)
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                        .map(get_ndvi)
                        .map(get_nbr)
                        .select(['NDVI', 'NBR']))
        

        # Extract NDVI and NBR means and date as Feature
        def reduce_img(img):
            stats = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=roi,
                scale=10,
                maxPixels=1e9
            )
            return ee.Feature(None, {
                'date': img.date().format('YYYY-MM-dd'),
                'NDVI': stats.get('NDVI'),
                'NBR': stats.get('NBR')
            })

        # Computes mean NDVI and NBR values for each image
        features = collection.map(reduce_img)

        # Extracts date, NDVI, and NBR values
        stats_list = features.aggregate_array('date').getInfo()
        ndvi_values = features.aggregate_array('NDVI').getInfo()
        nbr_values = features.aggregate_array('NBR').getInfo()

        # Arguments to smooth curves
        window = 19     
        poly = 6       

        # Smooth curves
        ndvi_smooth = savgol_filter(ndvi_values, window_length=window, polyorder=poly)
        nbr_smooth  = savgol_filter(nbr_values,  window_length=window, polyorder=poly)


        # Convert to datetime
        dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in stats_list]

        # Plot NDVI and NBR curves in the same figure
        fig = go.Figure()

        add_trace(fig, dates, ndvi_values, 'NDVI', line_color='green', marker_color='green')
        add_trace(fig, dates, ndvi_smooth, 'NDVI (smoothed)', line_color='green', marker_color='green', mode = 'lines')
        
        add_trace(fig, dates, nbr_values, 'NBR', line_color='red', marker_color='red')
        add_trace(fig, dates, nbr_smooth, 'NBR (smoothed)', line_color='red', marker_color='red', mode= "lines")


        # Add vertical line at 2025-01-07
        fig.add_vline(
            x=datetime.datetime(2025, 1, 7).timestamp() * 1000,  # Convert to milliseconds since epoch
            line_width=2,
            line_dash="dash",
            line_color="blue",
            annotation_text="First Fire detected",
            annotation_position="top right"
        )

        # Configures layout
        fig.update_layout(
            title='NDVI and NBR TimeSeries',
            xaxis_title='Date',
            yaxis_title='Index Value',
            xaxis=dict(showgrid=True),
            yaxis=dict(showgrid=True, title='Index Value'),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            template='plotly_white',
            hovermode='x unified'
        )

        # Save plot
        os.makedirs("results", exist_ok=True)
        fig.write_html("results/ndvi_nbr_plot.html")
        fig.write_image("results/ndvi_nbr_plot.pdf")
        
        print("Graphic saved at 'results/ndvi_nbr_plot.html'")
        fig.show(renderer = "browser")

    except Exception as e:
        print(f"Error in ndvi_nbr_plot: {e}")


In [15]:
curve_start_date = '2024-10-01'
curve_end_date = '2025-04-01'

# Run the function
ndvi_nbr_plot(curve_start_date, curve_end_date)

Error in ndvi_nbr_plot: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido



In [21]:
# GET IMAGES AFTER FIRE AND TODAY'S DATE

# After images
pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']

# Get most recent date
recent_date = get_most_recent_date('2025-04-01')
today = datetime.date.today().isoformat()

# Today images
today_result = get_all_collections(recent_date, today)
today_image = today_result['image']

print(f"The most recent date is {recent_date} for a cloud coverage of 10% or less.")

# Split map
left_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')
right_layer = geemap.ee_tile_layer(today_image, rgb_vis, 'Today')

Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Add layers  
Map.addLayer(pos_image, rgb_vis, 'After Fire', False)
Map.addLayer(today_image, rgb_vis, 'Today', False)



#==============================================
# Export images to Google Drive (uncomment to use)
#==============================================


# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")


# Add map control
antes_html, depois_html = create_labels_html('After', 'Today')
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

Map

The most recent date is 2025-12-28 for a cloud coverage of 10% or less.


Map(bottom=3347999.0, center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_tex…